In [73]:
import random
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service

class Survey:
    def __init__(self, webdriver_path, form_url, target_response):
        self.webdriver_path = webdriver_path
        self.form_url = form_url
        self.target_response = target_response
        self.complete = 0

        # Initialize the browser
        service = Service(webdriver_path)
        self.driver = webdriver.Chrome(service=service)
        #self.driver.maximize_window()

    @staticmethod
    def random_text():
        """Generate random text for form fields."""
        return random.choice(['Yes', 'No', 'Probably', 'Maybe', 'Not sure'])

    def fill_form(self):
        """Fill out the form fields automatically based on their type."""
        try:

            # Handle radio group questions
            # radiogroups = self.driver.find_elements(By.XPATH, '//div[@role="radiogroup"]')
            #
            # for radiogroup in radiogroups:
            #     options = radiogroup.find_elements(By.XPATH, './/div[@role="radio"]')
            #     if options:
            #         random.choice(options).click()

            radiogroups = self.driver.find_elements(By.XPATH, '//div[@role="radiogroup"]')

            desired_answers = [
                "3",  # Đáp án mong muốn cho câu hỏi 1
                "Cột 3, câu trả lời cho Hàng 3",
                "Cột 3, câu trả lời cho Hàng 2",
                "Cột 3, câu trả lời cho Hàng 1",
                'Tùy chọn 2'
            ]

            for idx, radiogroup in enumerate(radiogroups):
                options = radiogroup.find_elements(By.XPATH, './/div[@role="radio"]')

                if options:
                    selected_option = None
                    for option in options:
                        if option.get_attribute("aria-checked") == "true":
                            selected_option = option
                            break

                    if not selected_option:
                        valid_answers = {option.get_attribute("aria-label"): option for option in options if option.get_attribute("aria-label") in desired_answers}

                        #for valid_answer in valid_answers.values():
                            #valid_answer.click()
                        if valid_answers:
                            first_valid_answer = next(iter(valid_answers.values()))
                            first_valid_answer.click()

                        else:
                            random_option = random.choice(options)
                            random_option.click()

            # Handle checkbox questions
            checkboxes = self.driver.find_elements(By.XPATH, '//div[@role="listitem"]')

            for checkbox_group in checkboxes:
                checkboxes = checkbox_group.find_elements(By.XPATH, './/div[@role="checkbox"]')
                if checkboxes:
                    random.choice(checkboxes).click()

            # Handle short text fields
            short_text_fields = self.driver.find_elements(By.XPATH, '//input[@type="text"]')

            for short_field in short_text_fields:
                try:
                    parent = short_field.find_element(By.XPATH, './ancestor::div[@role="radiogroup"]')
                except:
                    parent = None
                if parent and len(desired_answers) != 0:
                    continue
                short_field.send_keys(self.random_text())

            # Handle long text fields
            long_text_fields = self.driver.find_elements(By.XPATH, '//textarea')

            for long_field in long_text_fields:
                try:
                    parent = long_field.find_element(By.XPATH, './ancestor::div[@role="radiogroup"]')
                except:
                    parent = None
                if parent and len(desired_answers) != 0:
                    continue
                long_field.send_keys(self.random_text())

            # Handle dropdown questions
            drop_groups = self.driver.find_elements(By.XPATH, '//div[@role="presentation"]')

            if drop_groups:
                for drop_group in drop_groups:
                    try:
                        drop_group.click()
                        time.sleep(0.3)
                        select_option = None
                        select_option = drop_group.find_elements(By.XPATH, './/div[@role="option" and @aria-selected="false"]')
                        if select_option and isinstance(select_option, list):
                            WebDriverWait(self.driver, 10).until(EC.element_to_be_clickable(select_option))
                            random.choice(select_option).click()
                    except Exception as e:
                        print(f"Error: {e}")

            time.sleep(0.2)  # Delay between processing each question

        except Exception as e:
            print(f"Error while filling form: {e}")

    def click_next(self):
        """Click the 'Next' button if available."""
        try:
            next_button = self.driver.find_element(By.XPATH, '//span[contains(text(), "Tiếp")]')
            WebDriverWait(self.driver, 10).until(EC.element_to_be_clickable(next_button))
            next_button.click()
            self.driver.execute_script("document.body.style.zoom='50%'")
            time.sleep(1)
            return True
        except Exception as e:
            print(f"Error while clicking next: {e}")
            return False

    def submit_form(self):
        """Click the 'Submit' button to submit the form."""
        try:
            submit_button = self.driver.find_element(By.XPATH, '//span[contains(text(), "Gửi")]')
            WebDriverWait(self.driver, 10).until(EC.element_to_be_clickable(submit_button))
            submit_button.click()
            self.driver.execute_script("document.body.style.zoom='50%'")
            self.complete += 1
            time.sleep(1)  # Wait for page reload after submission
        except Exception as e:
            print(f"Error while submitting form: {e}")

    def start(self):
        """Start the survey automation."""
        try:
            while self.complete < self.target_response:
                self.driver.get(self.form_url)
                self.driver.execute_script("document.body.style.zoom='50%'")
                time.sleep(1)  # Wait for the page to load

                # Lặp để điền form và bấm 'Next' đến khi không còn nút 'Next'
                while True:
                    self.fill_form()  # Gọi hàm điền form
                    if self.click_next():  # Nếu bấm được nút 'Next'
                        time.sleep(1)  # Đợi trang kế tiếp tải
                    else:
                        # Nếu không có nút 'Next', bấm 'Submit' và thoát vòng lặp
                        self.submit_form()
                        break

                print(f"Form submitted. Total responses submitted: {self.complete}/{self.target_response}")

            print(f"Task complete. Total responses submitted: {self.complete}/{self.target_response}")
        except Exception as e:
            print(f"An error occurred: {e}")
        finally:
            try:
                self.driver.quit()
            except Exception as quit_error:
                print(f"Browser already closed. Clean exit. ({quit_error})")


In [74]:
# Khởi tạo và chạy Survey
webdriver_path1 = "data/chromedriver.exe"  # Đường dẫn đến ChromeDriver
form_url1 = "https://forms.gle/g1v3u2oZsC2A6a2w6"  # URL Google Form
target_response1 = 1  # Số lượng phản hồi mục tiêu

survey = Survey(webdriver_path1, form_url1, target_response1)
survey.start()


Error: Message: element click intercepted: Element <div class="ry3kXd" jsname="d9BH4c" role="presentation" aria-hidden="true">...</div> is not clickable at point (408, 196). Other element would receive the click: <div class="MocG8c HZ3kWc mhLiyf OIC90c LMgvRb" jsname="wQNmvb" jsaction="" data-value="C" aria-selected="false" role="option" tabindex="-1">...</div>
  (Session info: chrome=131.0.6778.265)
Stacktrace:
	GetHandleVerifier [0x00007FF6A23D80D5+2992373]
	(No symbol) [0x00007FF6A206BFD0]
	(No symbol) [0x00007FF6A1F0590A]
	(No symbol) [0x00007FF6A1F60F2E]
	(No symbol) [0x00007FF6A1F5E9CC]
	(No symbol) [0x00007FF6A1F5BBA6]
	(No symbol) [0x00007FF6A1F5AB01]
	(No symbol) [0x00007FF6A1F4CD40]
	(No symbol) [0x00007FF6A1F7F36A]
	(No symbol) [0x00007FF6A1F4C596]
	(No symbol) [0x00007FF6A1F7F580]
	(No symbol) [0x00007FF6A1F9F584]
	(No symbol) [0x00007FF6A1F7F113]
	(No symbol) [0x00007FF6A1F4A918]
	(No symbol) [0x00007FF6A1F4BA81]
	GetHandleVerifier [0x00007FF6A2436A2D+3379789]
	GetHandleVe